In [ ]:
import numpy as np
from pathlib import Path    

data_path = Path("..") / "data" / "point_cloud_20260204T140758Z.npz"
data = np.load(data_path)

print(data.files)

['points', 'width', 'height', 'depth_scale', 'fx', 'fy', 'cx', 'cy', 'intr_width', 'intr_height']


array([[-0., -0.,  0.],
       [-0., -0.,  0.],
       [-0., -0.,  0.],
       ...,
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.]], dtype=float32)

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Use the points array from the loaded npz
points = data["points"] if "points" in data.files else data[data.files[0]]

# Clean and optionally downsample for responsiveness
mask = np.isfinite(points).all(axis=1)
pts = points[mask]
pts = pts[~np.all(pts == 0, axis=1)]

max_points = 200000
if pts.shape[0] > max_points:
    rng = np.random.default_rng(0)
    idx = rng.choice(pts.shape[0], max_points, replace=False)
    pts = pts[idx]

fig = go.Figure(data=[go.Scatter3d(
    x=pts[:, 0], y=pts[:, 1], z=pts[:, 2],
    mode='markers',
    marker=dict(size=1, color=pts[:, 2], colorscale='Viridis', opacity=0.7)
)])
fig.update_layout(
    title='Point Cloud',
    scene=dict(aspectmode='data'),
    margin=dict(l=0, r=0, b=0, t=30)
)
fig.show()
